# Profitable App Profiles for the App Store and Google Play Markets

For this project, we take the position of a data analyst working for a company that builds Android and iOS mobile apps. We make our apps available on Google Play and the App Store.

We only build apps that are free to download and install, and our main source of revenue consists of in-app ads. This means that the number of users of our apps determines our revenue for any given app - the more users who see and engage with the ads, the better. Our goal for this project is to analyze data to help our developers understand what kind of apps are likely to attract more users.

## Opening and Exploring the Data

As of September 2018 there were approximately 2 million iOS apps available on the App Store, and 2.1 million Android apps on Google Play.

![img](https://s3.amazonaws.com/dq-content/350/py1m8_statista.png) Source: [Statista](https://www.statista.com/statistics/276623/number-of-apps-available-in-leading-app-stores/) 

Collecting data for over four million apps requires a significant amount of time and money, so we'll try to analyze a sample of the data instead. To avoid spending resources with collecting new data ourselves, we should first try to see whether we can find any relevant existing data at no cost. Fortunately, there are two data sets that seem suitable for our purposes.

* [A data set](https://www.kaggle.com/datasets/lava18/google-play-store-apps) containing data approximately ten thousand Android apps from Google Play.
* [A data set](https://www.kaggle.com/datasets/ramamet4/app-store-apple-data-set-10k-apps) containing approximately seven thousand iOS apps from the App Store.

Let's start by opening the two data sets and then continue with exploring the data.

In [1]:
from csv import reader

# The Google Play data set
google = list(reader(open('../data_sets/app_profiles/googleplaystore.csv', encoding='utf8')))
google_header = google[0]
google = google[1:]

# The App Store data set
ios = list(reader(open('../data_sets/app_profiles/AppleStore.csv', encoding='utf8')))
ios_header = ios[0]
ios = ios[1:]

To make it easier to explore the two data sets, we'll first write a function named `explore_data()` that we can use repeatedly to explore rows in a more readable way. We'll also add an option for our function to show the number of rows and columns for any data set.

In [2]:
def explore_data(dataset, start=0, end=5, rows_and_columns=False):
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row)
        print('\n') # adds empty line after each row for easy reading

    if rows_and_columns:
        print('Number of rows: ', len(dataset))
        print('Number of columns: ', len(dataset[0]))

In [3]:
print('Google Play Apps', '\n')
print(google_header, '\n')
explore_data(google, 0, 3, True)

Google Play Apps 

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'] 

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


Number of rows:  10841
Number of columns:  13


We can see the Google Play data set has 10841 apps and 13 columns. At a quick glance, the columns that might be useful for the purpose of our analysis are `'App'`, `'Category'`, `'Reviews'`, `'Installs'`, `'Type'`, `'Price'`, and `'Genres'`.

Now let's take a look at the App Store data set.

In [4]:
print('iOS Apps', '\n')
print(ios_header, '\n')
explore_data(ios, 0, 3, True)

iOS Apps 

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic'] 

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


Number of rows:  7197
Number of columns:  16


Here the App Store data set has 7198 apps and 16 columns. The columns that look like they will be most useful for us will be `'track_name'`, `'currency'`, `'ratings_count_tot'`, `'ratings_count_ver'`, and `'prime_genre'`.

## Deleting Wrong Data

The Google Play data set has a dedicated [discussion section](https://www.kaggle.com/datasets/lava18/google-play-store-apps/discussion), and we can see that [one of the discussions](https://www.kaggle.com/datasets/lava18/google-play-store-apps/discussion/66015) outlines an error for row 10472. Let's print this row and compare it against the header and another row that is correct.

In [5]:
print(google[10472])  # incorrect row
print('\n')
print(google_header)  # header
print('\n')
print(google[0])      # correct row

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


The row 10472 corresponds to the app *Life Made WI-Fi Touchscreen Photo Frame*, and we can see that the rating is 19. This is clearly off because the maximum rating for a Google Play app is 5. It appears to be a consequence of a missing `'Category'` column, which transposed everything to the left once. This also means that the row has an incorrect number of columns.

We create a function `'clean_incorrect_columns()'` to iterate over the dataset and delete any rows that don't have the same number of columns as the header.

In [6]:
def clean_incorrect_columns(header, dataset):
    clean_dataset = []

    for row in dataset:
        if len(row) != len(header):
            continue
        clean_dataset.append(row)

    return clean_dataset

In [7]:
print(f'Google dataset contains {len(google)} entries pre-cleaning')
google = clean_incorrect_columns(google_header, google)
print(f'Google dataset contains {len(google)} entries post-cleaning')
print('\n')
print(f'iOS dataset contains {len(ios)} entries pre-cleaning')
ios = clean_incorrect_columns(ios_header, ios)
print(f'iOS dataset contains {len(ios)} entries post-cleaning')


Google dataset contains 10841 entries pre-cleaning
Google dataset contains 10840 entries post-cleaning


iOS dataset contains 7197 entries pre-cleaning
iOS dataset contains 7197 entries post-cleaning


We can see that the iOS data set had no problems, and the only column with problems in the Google Play data set was the one we already identified.

## Removing Duplicate Entries

### Part One

If we explore the Google Play data set long enough, we'll find that some apps have more than one entry. For instance, the application Instagram has four entries:

In [8]:
for app in google:
    name = app[0]
    if name == 'Instagram':
        print(app)

['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


We want to know how many cases there are where an app is counted more than once, so we create the function `'get_duplicate_apps()'` to count them for us. This function returns a list of all instances of apps that have already appeared elsewhere in the data set.

In [9]:
def get_duplicate_apps(dataset, name_index):
    unique_app = []
    dup_app = []
    
    for row in dataset:
        if row[name_index] in unique_app:
            dup_app.append(row[name_index])
            continue
        unique_app.append(row[name_index])

    return dup_app

In [10]:
google_duplicates = get_duplicate_apps(google, 0)
ios_duplicates = get_duplicate_apps(ios, 0)

print(f'Google duplicate apps count: {len(google_duplicates)}', '\n')
print('Examples of duplicate apps:', google_duplicates[:15], '\n')
print(f'iOS duplicate apps list: {len(ios_duplicates)}', '\n')



Google duplicate apps count: 1181 

Examples of duplicate apps: ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack', 'FreshBooks Classic', 'Insightly CRM', 'QuickBooks Accounting: Invoicing & Expenses', 'HipChat - Chat Built for Teams', 'Xero Accounting Software'] 

iOS duplicate apps list: 0 



We don't want to count any given app more than once when we analyze the data, so we need to remove all duplicate entries and keep only one entry per app. One thing we could do is remove the duplicate rows arbitrarily, but we can do better.

If you examine the rows we printed up above for the Instagram app, the main difference happens in the fourth position of each row, which corresponds to the number of reviews. The different numbers imply that the data was collected at different times. We can use this to build a criterion for keeping rows. We don't remove rows arbitrarily, but instead will keep the rows that have the highest number of reviews because the higher the number of reviews, the more reliable the ratings.

The below function takes the dataset, the index of the name, and the index of the review count and returns a cleaned dataset. It works by first adding all of the apps that aren't present in the list of duplicates to a fresh list, and then grabbing all instances of the duplicated apps that have the highest review count. There are some apps with multiple entries with the highest reviews (as we can see from the small selection above), and any of them will work. We grab the last entry in the list for convenience.

We then check to make sure we removed all duplicates by checking the difference between the size of the old datasets and the size of the new datasets.

In [11]:
def clean_duplicate_data(dataset, name_index, review_index):
    duplicates_set = set(get_duplicate_apps(dataset, name_index))
    clean_dataset = [row for row in dataset if row[name_index] not in duplicates_set]
    
    for app_name in duplicates_set:
        entry_list = [row for row in dataset if row[name_index] == app_name]

        max_count_entries = [row for row in dataset if float(row[review_index]) == max([float(row[review_index]) for row in entry_list]) and row[name_index] == app_name]
        clean_dataset.append(max_count_entries.pop())

    return clean_dataset
                

In [12]:
old_google_len = len(google)
google = clean_duplicate_data(google, 0, 3)
new_google_len = len(google)
old_ios_len = len(ios)
ios = clean_duplicate_data(ios, 0, 5)
new_ios_len = len(ios)

print(f'Google cleaned rows: {old_google_len - new_google_len}')
print(f'iOS cleaned rows: {old_ios_len - new_ios_len}')

Google cleaned rows: 1181
iOS cleaned rows: 0


The number of removed rows match the counts we got above for duplicate entries, so it is safe to say we got all of the duplicates and now have a dataset where each app is unique.

## Removing Non-English Apps

### Part One

If you explore the data sets enough, you'll notive the names of some of the apps suggest they are not directed towards an English-speaking audience. Below, we see a couple of examples from both data sets:

In [13]:
print(ios[813][1])
print(ios[1193][1])
print(google[225][0])
print(google[428][0])

爱奇艺PPS -《欢乐颂2》电视剧热播
聚力视频HD-人民的名义,跨界歌王全网热播
Truyện Vui Tý Quậy
Flame - درب عقلك يوميا


We only want to consider apps for an English-speaking audience, so next we'll go about removing any apps designed for another audience. A simple way to do this is by removing any apps that don't use common english characters -- English text usually includes letter from the English alphabet, numbers composed of single digits from 0 to 9, punctuation marks (., !, ?, ;, etc.), and other symbols (+, *, /, etc.)

All these characters that are specific to English texts are encoded using the ASCII standard. Each ASCII character has a corresponding number between 0 and 127 associated with it, and we can take advantage of that the build a function that checks an app name and tells us whether it contains non-ASCII characters.

We build this function below, and we use the built-in `ord()` function to find out the correspondingn encoding number of each character.

In [14]:
def is_english(name):
    for char in name:
        if ord(char) > 127:
            return False
    return True

In [15]:
print(is_english('Instagram'))
print(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))

True
False


The function seems to work fine, but some English app names use emojis or other symbols (™, — (em dash), – (en dash), etc.) that all fall outside of the ASCII range. Because of this, we'll have a number of false positives in its current form.

In [16]:
print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))

print(ord('™'))
print(ord('😜'))

False
False
8482
128540


### Part Two

To minimize the impact of data loss, we'll only remove an app if its name has more than three non-ASCII characters:

In [17]:
def is_english(name):
    non_ascii = 0

    for character in name:
        if ord(character) > 127:
            non_ascii += 1

    if non_ascii > 3:
        return False
    return True

print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))

True
True


The function is not perfect, and some non-English apps might get past our filter, but this seems good enough at this point in our analysis — we shouldn't spend too much time on optimization at this point.

Below, we use the `is_english()` function to filter out the non-English apps for both data sets:

In [18]:
clean_google = [row for row in google if is_english(row[0])]
clean_ios = [row for row in ios if is_english(row[1])]

In [19]:
print('Google dataset all language \n')
explore_data(google, start=0, end=0, rows_and_columns=True)
print('\n')
print('Google dataset English only \n')
explore_data(clean_google, start=0, end=0, rows_and_columns=True)
print('\n')
print('iOS dataset all language \n')
explore_data(ios, start=0, end=0, rows_and_columns=True)
print('\n')
print('iOS dataset English only \n')
explore_data(clean_ios, start=0, end=0, rows_and_columns=True)


Google dataset all language 

Number of rows:  9659
Number of columns:  13


Google dataset English only 

Number of rows:  9614
Number of columns:  13


iOS dataset all language 

Number of rows:  7197
Number of columns:  16


iOS dataset English only 

Number of rows:  6183
Number of columns:  16


It looks like we were able to discard about 60 apps from Google Play and 1000+ from the Apple Store. This isn't particularly relevant to our research, but based on this it appears that Apple has a much larger non-English market share.

We're left with 9614 Android apps and 6183 iOS apps.

## Isolating the Free Apps

As we mentioned in the introduction, we only build apps that are free to download and install, and our main source of revenue consists of in-app ads. Our data sets contain both free and non-free apps, and we'll need to isolate only the free apps for our analysis. Below, we isolate the free apps for both our data sets.

In [20]:
def is_free(row, index):
    return float(row[index].strip('$')) == 0

def clean_free_apps(dataset, index):
    clean_dataset = []
    for row in dataset:
        if is_free(row, index):
            clean_dataset.append(row)

    return clean_dataset

In [21]:
free_google = clean_free_apps(clean_google, 7)
free_ios = clean_free_apps(clean_ios, 4)

In [22]:
print('Google dataset All Price \n')
explore_data(clean_google, start=0, end=0, rows_and_columns=True)
print('\n')
print('Google dataset Free Only \n')
explore_data(free_google, start=0, end=0, rows_and_columns=True)
print('\n')
print('iOS dataset All Price \n')
explore_data(clean_ios, start=0, end=0, rows_and_columns=True)
print('\n')
print('iOS dataset Free Only \n')
explore_data(free_ios, start=0, end=0, rows_and_columns=True)

Google dataset All Price 

Number of rows:  9614
Number of columns:  13


Google dataset Free Only 

Number of rows:  8864
Number of columns:  13


iOS dataset All Price 

Number of rows:  6183
Number of columns:  16


iOS dataset Free Only 

Number of rows:  3222
Number of columns:  16


We're left with 8864 Android apps and 3222 iOS apps, which should be enough for our analysis.

## Most Common Apps by Genre

### Part One

As mentioned in the introduction, our aim is to determine the kinds of apps that are likely to attract more users because our revenue is highly influenced by the number of people using our apps.

To minimize risk and overhead, our validation strategy for an app idea has three steps:

1. Build a minimum Android version of the app, and add it to Google Play
2. If the app has a good response from users, we develop it further
3. If the app is profitable after six months, we build an iOS version of the app and add it to the App Store

Our end goal is to have a version of the app on both Google Play and the App Store, so we want to find app profiles that are successful in both markets. For instance, a profile that might work well for both markets might be a productivity app that makes use of gamification.

Let's begin the analysis by getting a sense of the most common genres for each market. For this, we'll build a frequency table for the `prime_genre` column of the App Store data set, and the `Genres` and `Category` columns of the Google Play data set.

### Part Two

We'll build two functions we can use to analyze the frequency tables:
* One function to generate frequency tables that show percentages
* Another function that we can use to display the percentages in a descending order

In [23]:
def freq_table(dataset, index):
    table = {}
    total = 0
    
    for row in dataset:
        total += 1
        value = row[index]
        if value in table:
            table[row[index]] += 1
        else:
            table[row[index]] = 1

    table_percentages = {}
    for key in table:
        percentage = (table[key] / total) * 100
        table_percentages[key] = percentage

    return table_percentages

In [24]:
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

### Part Three

We start by examining the frequency table for the `prime_genre` column of the App Store data set.

In [25]:
print('iOs prime_genre: \n')
display_table(free_ios, ios_header.index('prime_genre'))

iOs prime_genre: 

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


We can see that among the free English apps, more than half (58.16%) are games. Entertainment apps are close to 8%, followed by photo and video apps, which are close to 5%. Only 3.66% of the apps are designed for education, followed by social networking apps which amount for 3.29% of the apps in our data set.

The general impression is that the App Store (at least the part containing free English apps) is dominated by apps that are designed for fun (games, entertainment, photo and video, social networking, sports, music, etc.), while apps with practical purposes (education, shopping, utilities, productivity, lifestyle, etc.) are more rare. However, the fact that fun apps are the most numerous doesn't also imply that they also have the greatest number of users — the demand might not be the same as the offer.

Let's continue by examining the `Genres` and `Category` columns of the Google Play data set (two columns which seem to be related).

In [27]:
print('Google Category: \n')
display_table(free_google, google_header.index('Category'))

Google Category: 

FAMILY : 18.94178700361011
GAME : 9.713447653429604
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.15072202166065
ENTERTAINMENT : 0.947653429602888
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
AR

The landscape seems significantly different on Google Play: there are not that many apps designed for fun, and it seems that a good number of apps are designed for practical purposes (family, tools, business, lifestyle, productivity, etc.). However, if we investigate this further, we can see that the family category (which accounts for almost 19% of the apps) mostly means games for kids.

![img](https://camo.githubusercontent.com/0d974d86dfcb791ca1d1505f810ba5a191bb6248e413d9b6ae1a60b211cca918/68747470733a2f2f73332e616d617a6f6e6177732e636f6d2f64712d636f6e74656e742f3335302f7079316d385f66616d696c792e706e67)

Even so, practical apps seem to have a better representation on Google Play compared to App Store. This picture is also confirmed by the frequency table we see for the `Genres` column.

In [28]:
print('Google Genres: \n')
display_table(free_google, google_header.index('Genres'))

Google Genres: 

Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.531137184115524
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2378158844765346
Action : 3.1024368231046933
Health & Fitness : 3.0798736462093865
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.8501805054151623
Video Players & Editors : 1.7712093862815883
Casual : 1.7599277978339352
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & Demo : 0.9363718411552346
Auto & Vehicles 

The difference between the `Genres` and the `Category` columns is not crystal clear, but one thing we can notice is that the `Genres` column is much more granular (it has more categories). We're only looking at the big picture at the moment, so we'll only work with the `Category` column moving forward.

Up to this point, we found that the App Store is dominated by apps designed for fun, while Google Play shows a more balanced landscape of both practical and for-fun apps. Now we'd like to get an idea about the kind of apps that have most users.

## Most Popular Apps by Genre on the App Store

One way to find out what genres are the most popular (have the most users) is to calculate the average number of installs for each app genre. For the Google Play data set, we can find this information in the `Installs` column, but for the App Store data set this information is missing. As a workaround, we'll take the total number of user ratings as a proxy, which we can find in the `rating_count_tot` column.

Below, we calculate the average number of user ratings per app genre in the App Store:

In [32]:
genre_idx = ios_header.index('prime_genre')
prime_genre_freq = freq_table(free_ios, genre_idx)

genre_list = []
for genre in prime_genre_freq:
    total = 0
    len_genre = 0
    for app in free_ios:
        if app[genre_idx] == genre:
            total += float(app[ios_header.index('rating_count_tot')])
            len_genre += 1

    genre_list.append((genre, round(total / len_genre, 3)))

genre_list.sort(key=lambda genre: genre[1], reverse=True)
print('Average number of reviews per genre: \n')
for genre in genre_list:
    print(f'{genre[0]}: {genre[1]}')

Average number of reviews per genre: 

Navigation: 86090.333
Reference: 79350.471
Social Networking: 71548.349
Music: 57326.53
Weather: 52279.893
Book: 46384.917
Food & Drink: 33333.923
Finance: 32367.029
Photo & Video: 28441.544
Travel: 28243.8
Shopping: 27230.735
Health & Fitness: 23298.015
Sports: 23008.899
Games: 22886.367
News: 21248.023
Productivity: 21028.411
Utilities: 19156.494
Lifestyle: 16815.48
Entertainment: 14195.359
Business: 7491.118
Education: 7003.983
Catalogs: 4004.0
Medical: 612.0


On average, navigation apps have the highest number of user reviews, but this figure is heavily influenced by Waze and Google Maps, which have close to half a million user reviews.

In [29]:
for app in free_ios:
    if app[-5] == 'Navigation':
        print(app[1], ':', app[5]) # print name and number of ratings

Waze - GPS Navigation, Maps & Real-time Traffic : 345046
Google Maps - Navigation & Transit : 154911
Geocaching® : 12811
CoPilot GPS – Car Navigation & Offline Maps : 3582
ImmobilienScout24: Real Estate Search in Germany : 187
Railway Route Search : 5


The same pattern applies to social networking apps, where the average number is heavily influenced by a few giants like Facebook, Pinterest, Skype, etc. Same applies to music apps, where a few big players like Pandora, Spotify, and Shazam heavily influence the average number.

Our aim is to find popular genres, but navigation, social networking, and music seem more popular than they actually are. A few big names dominate the space, and a new app would have a very difficult time breaking into any of those markets. We could get a better picture by removing individual apps whose review counts are significantly higher than the average and running the analysis again, but that is a level of detail we can get to later.

Reference apps have 79350 user ratings on average, but it's actually the Bible and Dictionary.com which skew up the average rating:

In [30]:
for app in free_ios:
    if app[-5] == 'Reference':
        print(f'{app[1]}: {app[5]}')

Bible: 985920
Dictionary.com Dictionary & Thesaurus: 200047
Dictionary.com Dictionary & Thesaurus for iPad: 54175
Google Translate: 26786
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran: 18418
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition: 17588
Merriam-Webster Dictionary: 16849
Night Sky: 12122
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE): 8535
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools: 4693
GUNS MODS for Minecraft PC Edition - Mods Tools: 1497
Guides for Pokémon GO - Pokemon GO News and Cheats: 826
WWDC: 762
Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free: 718
VPN Express: 14
Real Bike Traffic Rider Virtual Reality Glasses: 8
教えて!goo: 0
Jishokun-Japanese English Dictionary & Translator: 0


However, this niche shows some potential. One thing we could do is take another popular book and turn it into an app where we could add different features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes about the book, etc. On top of that, we could embed a dictionary within the app so users don't need to exit our app to look up words.

This idea seems to fit well within the fact that the App Store is dominated by for-fun apps. This suggests the market might be a bit saturated with for-fun apps, which means a practical app might have more of a chance to stand out among the huge number of apps on the App Store.

Other genres that seem popular include weather, book, food and drink, or finance. The book genre seems to overlap a bit with the app idea we described above, but the other genres don't seem too interesting to us:
* Weather apps: people generally don't spend too much time in-app, and the chances of making profit from in-app adds are low. Also, getting reliable live weather data may require us to connect our apps to non-free APIs.
* Food and drink: examples here include Starbucks, Dunkin' Donuts, McDonald's, etc., i.e. apps attached to food brands that already exist. This would require actual cooking and delivery service, which is outside the scope of our company.
* Finance apps: these apps involve banking, paying bills, money transfers, etc. Building a finance app requires domain knowledge, and we don't want to hire a finance expert just to build an app.

Now let's analyze the Google Play market a bit.

## Most Popular Apps by Genre on Google Play

Unlike the App Store data set, the Google Play data set is a bit more loose with its data. Instead of an exact number of downloads, Google Play gives a size bucket for the number of downloads as we can see here:

In [32]:
display_table(free_google, 5)

1,000,000+ : 15.726534296028879
100,000+ : 11.552346570397113
10,000,000+ : 10.548285198555957
10,000+ : 10.198555956678701
1,000+ : 8.393501805054152
100+ : 6.915613718411552
5,000,000+ : 6.825361010830325
500,000+ : 5.561823104693141
50,000+ : 4.7721119133574
5,000+ : 4.512635379061372
10+ : 3.5424187725631766
500+ : 3.2490974729241873
50,000,000+ : 2.3014440433213
100,000,000+ : 2.1322202166064983
50+ : 1.917870036101083
5+ : 0.78971119133574
1+ : 0.5076714801444043
500,000,000+ : 0.2707581227436823
1,000,000,000+ : 0.22563176895306858
0+ : 0.04512635379061372
0 : 0.01128158844765343


We can still perform analysis on the set, it will just be a bit less precise than if we had the exact numbers. We will use the size of the bucket for each app's download count. So all apps in the 10+ category will be treated as if they have 10 downloads, all apps in the 1,000+ category will be treated as if they have 1,000 downloads, etc. This will be enough information to give us the broad patterns that we're looking for.

We perform similar analysis as above, with the additional step of stripping the + sign from the data so that we can treat it as a number.

In [38]:
category_idx = google_header.index('Category')
reviews_freq = freq_table(free_google, category_idx)

genre_list = []
for category in reviews_freq:
    total = 0
    len_category = 0
    for app in free_google:
        if app[category_idx] == category:
            download_count = app[google_header.index('Installs')].strip('+').replace(',', '')
            total += float(download_count)
            len_category += 1

    genre_list.append((category, round(total / len_category, 3)))

genre_list.sort(key=lambda category: category[1], reverse=True)
print('Average number of reviews per genre: \n')
for genre in genre_list:
    print(f'{genre[0]}: {genre[1]}')

Average number of reviews per genre: 

COMMUNICATION: 38456119.167
VIDEO_PLAYERS: 24727872.453
SOCIAL: 23253652.127
PHOTOGRAPHY: 17840110.402
PRODUCTIVITY: 16787331.345
GAME: 15594505.749
TRAVEL_AND_LOCAL: 13984077.71
ENTERTAINMENT: 11719761.905
TOOLS: 10801391.299
NEWS_AND_MAGAZINES: 9549178.468
BOOKS_AND_REFERENCE: 8767811.895
SHOPPING: 7036877.312
PERSONALIZATION: 5201482.612
WEATHER: 5074486.197
HEALTH_AND_FITNESS: 4188821.985
MAPS_AND_NAVIGATION: 4056941.774
FAMILY: 3698568.011
SPORTS: 3638640.143
ART_AND_DESIGN: 1986335.088
FOOD_AND_DRINK: 1924897.736
EDUCATION: 1841666.667
BUSINESS: 1712290.147
LIFESTYLE: 1437816.269
FINANCE: 1387692.476
HOUSE_AND_HOME: 1331540.562
DATING: 854028.83
COMICS: 817657.273
AUTO_AND_VEHICLES: 647317.817
LIBRARIES_AND_DEMO: 638503.735
PARENTING: 542603.621
BEAUTY: 513151.887
EVENTS: 253542.222
MEDICAL: 120550.62


Much like with the App Store, communication apps are close to the top. Also like the App Store, they are at the top because of a short list of incredibly popular apps. If we just look at communication apps that have over 500 million downloads, we get the following list:

In [49]:
for app in free_google:
    if app[1] == 'COMMUNICATION' and (app[5] == '1,000,000,000+'
                                      or app[5] == '500,000,000+'):
        print(app[0], ':', app[5])

Viber Messenger : 500,000,000+
Messenger – Text and Video Chat for Free : 1,000,000,000+
UC Browser - Fast Download Private & Secure : 500,000,000+
Google Duo - High Quality Video Calls : 500,000,000+
LINE: Free Calls & Messages : 500,000,000+
Google Chrome: Fast & Secure : 1,000,000,000+
WhatsApp Messenger : 1,000,000,000+
Skype - free IM & video calls : 1,000,000,000+
Gmail : 1,000,000,000+
Hangouts : 1,000,000,000+
imo free video calls and chat : 500,000,000+


Apps like these skew the numbers in a way that obfuscates the information we want to find. Having a single app with over 100 million downloads can make a genre of app seem much more popular than it actually is when in many cases it isn't the genre so much as a select list of specific apps that are popular. If that is the case, then we would have nothing to gain trying to develop an app for that genre.

What would benefit us the most for our analysis is excluding any apps that get over 100 million downloads, so we only consider categories where small apps have a chance to make break into the market. We can do that for all of the categories and get a fresh list that is more useful to us.

In [51]:
category_idx = google_header.index('Category')
reviews_freq = freq_table(free_google, category_idx)

genre_list = []
for category in reviews_freq:
    total = 0
    len_category = 0
    for app in free_google:
        if app[category_idx] == category:
            download_count = app[google_header.index('Installs')].strip('+').replace(',', '')
            if float(download_count) >= 100000000:
                continue
            total += float(download_count)
            len_category += 1

    genre_list.append((category, round(total / len_category, 3)))

genre_list.sort(key=lambda category: category[1], reverse=True)
print('Average number of reviews per genre for apps with less than 100,000,000 downlaods: \n')
for genre in genre_list:
    print(f'{genre[0]}: {genre[1]}')

Average number of reviews per genre for apps with less than 100,000,000 downlaods: 

PHOTOGRAPHY: 7670532.293
GAME: 6267917.02
ENTERTAINMENT: 6132405.063
VIDEO_PLAYERS: 5544878.133
WEATHER: 5074486.197
SHOPPING: 4640920.541
COMMUNICATION: 3603485.388
PRODUCTIVITY: 3379657.319
TOOLS: 3191461.129
SOCIAL: 3084582.52
SPORTS: 2994082.552
TRAVEL_AND_LOCAL: 2944079.634
PERSONALIZATION: 2549775.832
MAPS_AND_NAVIGATION: 2484104.754
FAMILY: 2348285.7
HEALTH_AND_FITNESS: 2005713.661
ART_AND_DESIGN: 1986335.088
FOOD_AND_DRINK: 1924897.736
EDUCATION: 1841666.667
NEWS_AND_MAGAZINES: 1502841.878
BOOKS_AND_REFERENCE: 1437212.216
HOUSE_AND_HOME: 1331540.562
BUSINESS: 1226918.741
LIFESTYLE: 1152128.78
FINANCE: 1086125.786
DATING: 854028.83
COMICS: 817657.273
AUTO_AND_VEHICLES: 647317.817
LIBRARIES_AND_DEMO: 638503.735
PARENTING: 542603.621
BEAUTY: 513151.887
EVENTS: 253542.222
MEDICAL: 120550.62


Without the massive apps, the average has dropped quite a bit for a few categories. Communication is reduced by a factor of 10, from 38 million to 3.8 million. Similarly, video players is reduced by a factor of 5. Likely because of apps like YouTube and Google Play Movies & TV.

The game category seems popular, but, as we found when exploring the App Store data, the market for mobile games seems a bit saturated, so we'd like to come up with a different plan if possible.

The books and reference category still looks fairly popular, sitting right in the middle of the list after excluding all of the massive apps. The smaller apps still average over a million downloads. It's interesting to explore this data in more depth, because we found this genre has some potential to work well on the App Store, and our aim is to recommend an app genre that shows potential for being profitable on both the App Store and Google Play.

Let's take a look at some of the apps from this category and their number of installs:

In [53]:
for app in free_google:
    if app[1] == 'BOOKS_AND_REFERENCE' and app[5] not in ['1,000,000,000+', '500,000,000+']:
        print(app[0], ':', app[5])

E-Book Read - Read Book for free : 50,000+
Download free book with green book : 100,000+
Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Free Panda Radio Music : 100,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
English Grammar Complete Handbook : 500,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
AlReader -any text book reader : 5,000,000+
Offline English Dictionary : 100,000+
Offline: English to Tagalog Dictionary : 500,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
Recipes of Prophetic Medicine for free : 500,000+
ReadEra – free ebook reader : 1,000,000+
Anonymous caller detection : 10,000+
Litnet - E-books : 100,000+
Read books online : 5,000,000+
English to Urdu Dictionary : 500,000+
eBoox: book reader fb2 epub zip : 1,000,000+
English Persian Dictionary : 500,000+
Flybook : 500,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
English translation from Bengali : 100,000+
Pdf Book Download 

The books and reference category has a number of different apps: software for processing and reading ebooks, various collections of libraries, dictionaries, tutorials on programming or languages, etc. There are still a few very popular books that skew the data:

In [54]:
for app in free_google:
    if app[1] == 'BOOKS_AND_REFERENCE' and app[5] in ['1,000,000,000+', '500,000,000+','100,000,000+']:
        print(app[0], ':', app[5])

Google Play Books : 1,000,000,000+
Bible : 100,000,000+
Audiobooks from Audible : 100,000,000+
Amazon Kindle : 100,000,000+
Wattpad 📖 Free Books : 100,000,000+


However, it looks like there are only a few of them, so this market still shows some potential. Let's try to get some app ideas based on the kind of apps that are somewhere in the middle in terms of popularity (between 1,000,000 and 100,000,000 downloads):

In [55]:
for app in free_google:
    if app[1] == 'BOOKS_AND_REFERENCE' and app[5] in ['1,000,000+',
                                                      '5,000,000+',
                                                      '10,000,000+',
                                                      '50,000,000+',
                                                      '100,000,000+']:
        print(app[0], ':', app[5])

Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
AlReader -any text book reader : 5,000,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
ReadEra – free ebook reader : 1,000,000+
Read books online : 5,000,000+
eBoox: book reader fb2 epub zip : 1,000,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
Moon+ Reader : 10,000,000+
English-Myanmar Dictionary : 1,000,000+
Golden Dictionary (EN-AR) : 1,000,000+
All Language Translator Free : 1,000,000+
Bible : 100,000,000+
Aldiko Book Reader : 10,000,000+
Dictionary - WordWeb : 5,000,000+
50000 Free eBooks & Free AudioBooks : 5,000,000+
Al-Quran (Free) : 10,000,000+
Al Quran Indonesia : 10,000,000+
Al'Quran Bahasa Indonesia : 10,000,000+
Al Quran Al karim : 1,000,000+
Al Quran : EAlim - Translations & MP3 Offline : 5,000,000+
Koran Read &MP3 30 Juz Offline : 1,000,000+
Hafizi

This niche seems to be dominated by software for processing and reading ebooks, as well as various collections of libraries and dictionaries, so it's probably not a good idea to build similar apps since there will be some significant competition.

We also notice that there are quite a few apps built around the Quran, which suggests that building an app around a popular book can be profitable. It seems that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the App Store markets.

However, it looks like the market is already full of libraries, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version from the book, quizzes on the book, a forum where people can discuss the book, etc.

## Conclusions

In this project we analyzed data about the App Store and Google Play mobile apps with the goal of recommending anj app that could potentially be profitable on for both markets.

We concluded that taking a popular book and turning it into an app could be profitable for both the Google Play and the App Store markets. The markets are already full of libraries, so in order to garner attention we would need to add some additional features to draw users. Features such as daily quotes, quizzes, an audio version, or discussion forums.